<a href="https://colab.research.google.com/github/WayneGretzky1/nba_game_predictor/blob/main/nba_game_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

nba_df = pd.read_csv("https://raw.githubusercontent.com/WayneGretzky1/CSCI-4521-Applied-Machine-Learning/refs/heads/main/cleaned.csv")
nba_df.head()
